In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-12 14:50:53--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-12-12 14:50:53 (19.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"For input {context} target is {target}")

For input tensor([18]) target is 47
For input tensor([18, 47]) target is 56
For input tensor([18, 47, 56]) target is 57
For input tensor([18, 47, 56, 57]) target is 58
For input tensor([18, 47, 56, 57, 58]) target is 1
For input tensor([18, 47, 56, 57, 58,  1]) target is 15
For input tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
For input tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [12]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"For input {context.tolist()} the target is {target}")

For input [24] the target is 43
For input [24, 43] the target is 58
For input [24, 43, 58] the target is 5
For input [24, 43, 58, 5] the target is 57
For input [24, 43, 58, 5, 57] the target is 1
For input [24, 43, 58, 5, 57, 1] the target is 46
For input [24, 43, 58, 5, 57, 1, 46] the target is 43
For input [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
For input [44] the target is 53
For input [44, 53] the target is 56
For input [44, 53, 56] the target is 1
For input [44, 53, 56, 1] the target is 58
For input [44, 53, 56, 1, 58] the target is 46
For input [44, 53, 56, 1, 58, 46] the target is 39
For input [44, 53, 56, 1, 58, 46, 39] the target is 58
For input [44, 53, 56, 1, 58, 46, 39, 58] the target is 1
For input [52] the target is 58
For input [52, 58] the target is 1
For input [52, 58, 1] the target is 58
For input [52, 58, 1, 58] the target is 46
For input [52, 58, 1, 58, 46] the target is 39
For input [52, 58, 1, 58, 46, 39] the target is 58
For input [52, 58, 1, 58, 46, 39, 

In [13]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [15]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next = torch.multinomial(probs,num_samples=1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [16]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [18]:
batch_size = 32

for step in range(10000):
  xb,yb = get_batch('train')
  logits, loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.580815076828003


In [19]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


I weefapsshe d.
S dorithe bld avo the f pir inithe sengr benklos h O:


Foveplod? l lll tt brackinds shont t;


Be ivervin pishatill yo atel, th al.

MALI n'ghtho bar ker ve he ie arthe psed, mar wal th I Int whate RConiry l halean; m fo d pu wered Jume ave is nds:
OVVARd tor ave th g'eritNDYome pouk torithe, melu
Youn, drrowintherinche's d t nd alf d'sss:

S:
Twe tousthinon t, anthard wo amy, thir lame not
CAne g gons,
My ye hoTolat wiohes,
Thint ss hmiqur te Inde avite alisety; ay AD:

LI l I'


## Trick for Self Attention

In [20]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [21]:
# x[b,t] = mean{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) #bag of words
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev,0)

In [24]:
x[0],xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [28]:
# Version 2
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1,keepdim=True)
xbow2 = torch.zeros_like(xbow)
for b in range(B):
    xbow2[b] = wei @ x[b]
torch.allclose(xbow,xbow2)

True

In [31]:
# Version 3 using softmax

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei, dim=-1)       # (T, T)
xbow3 = torch.zeros_like(xbow)
for b in range(B):
    xbow3[b] = wei @ x[b]
torch.allclose(xbow,xbow3)

True

In [32]:
# Version 4 self attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

#Taking single head for self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1) # (B,T,C) @ (B,C,T) = (B,T,T)
tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei, dim=-1)       # (T, T)

v = value(x)
out = wei @ v
#out = wei @ x
out.shape

torch.Size([4, 8, 16])

In [33]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1)*head_size**-0.5 #scaled dot-product attention

In [34]:
k.var()

tensor(1.0449)

In [35]:
q.var()

tensor(1.0700)